# Generating Jacccard distance matrices efficiently

A parallelized appraoch inspired by http://na-o-ys.github.io/others/2015-11-07-sparse-vector-similarities.html

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import scipy

from sklearn import metrics as skm
import sklearn as sk

In [2]:
example_data = dict(
    Sample_1 = ["a", "b", "c", "d"],
    Sample_2 = ["a", "b", "c"],
    Sample_3 = ["c", "d", "e", "f", "g"],
    Sample_4 = ["e", "f", "g"],
)

In [3]:
example_df = pd.DataFrame.from_dict(example_data, orient="index").T
example_df.head()

,Sample_1,Sample_2,Sample_3,Sample_4
0,a,a,c,e
1,b,b,d,f
2,c,c,e,g
3,d,None,f,None
4,None,None,g,None


In [9]:
ex_df = pd.read_csv("../mon_num.csv")
ex_df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (26,139,574,578,863,917,970,991,1213,1214,1268,1416,1538,1583,1694,1999,2000,2001,2003,2004,2005,2353,2557,2558,2559,2569,2648,2661,2720) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,PRNP,PRODH,PROK2,PROKR2,PROM1,PROP1,PRPF3,PRPF31,PRPF4,PRPF6,...,PRKAR1A,PRKCD,PRKCG,PRKCSH,PRKD1,PRKDC,PRKG1,PRKN,PRKRA,PRMT7
0,4305.0,2353.0,118.0,118.0,7814.0,4322.0,7737.0,7737.0,7703.0,7703.0,...,4490.0,2716.0,2600.0,2617.0,2209.0,4430.0,12727.0,4409.0,4305.0,4322.0
1,2922.0,2133.0,4409.0,4409.0,7793.0,9888.0,7703.0,7703.0,987.0,987.0,...,2818.0,4315.0,4373.0,2239.0,963.0,2783.0,2705.0,2578.0,9763.0,8947.0
2,2549.0,750.0,4349.0,4374.0,7722.0,2920.0,987.0,987.0,980.0,980.0,...,2640.0,4313.0,2354.0,2094.0,958.0,9879.0,2875.0,2529.0,2544.0,9891.0
3,2533.0,733.0,9804.0,4349.0,7704.0,2750.0,7675.0,7675.0,7675.0,7675.0,...,4322.0,2719.0,2073.0,2093.0,750.0,12444.0,3549.0,2378.0,2451.0,12745.0
4,2529.0,729.0,2761.0,4322.0,7703.0,2615.0,7663.0,7663.0,662.0,7663.0,...,9830.0,2725.0,2070.0,2086.0,691.0,2079.0,2686.0,2360.0,2317.0,12448.0


In [10]:
ex_df = ex_df.iloc[:,1:10]
ex_df.head()

,PRODH,PROK2,PROKR2,PROM1,PROP1,PRPF3,PRPF31,PRPF4,PRPF6
0,2353.0,118.0,118.0,7814.0,4322.0,7737.0,7737.0,7703.0,7703.0
1,2133.0,4409.0,4409.0,7793.0,9888.0,7703.0,7703.0,987.0,987.0
2,750.0,4349.0,4374.0,7722.0,2920.0,987.0,987.0,980.0,980.0
3,733.0,9804.0,4349.0,7704.0,2750.0,7675.0,7675.0,7675.0,7675.0
4,729.0,2761.0,4322.0,7703.0,2615.0,7663.0,7663.0,662.0,7663.0


In [11]:
ex_df.shape

(8650, 9)

In [79]:
from scipy.spatial import distance as scd

In [84]:
scd.jaccard([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0])

0.2857142857142857

In [71]:
d = ex_df.to_dict(orient="series")
d = {k: v.dropna().apply(lambda x: "HP:" + str(x).split(".")[0]).tolist() for k, v in d.items()}

In [91]:
all_items = set()
for k, v in d.items():
    all_items = all_items | set(v)
all_items = pd.Series(list(all_items))

In [93]:
binarized_data = dict()
for k, v in d.items():
    binarized_data[k] = [1 if e in v else 0 for e in all_items]
binarized_df = pd.DataFrame(binarized_data)
binarized_df.head()

,PRODH,PROK2,PROKR2,PROM1,PROP1,PRPF3,PRPF31,PRPF4,PRPF6
0,0,0,1,0,1,0,0,0,0
1,0,0,0,1,0,1,1,1,1
2,0,1,1,0,1,0,0,0,0
3,0,1,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0


In [98]:
list(binarized_df.columns)

['PRODH',
 'PROK2',
 'PROKR2',
 'PROM1',
 'PROP1',
 'PRPF3',
 'PRPF31',
 'PRPF4',
 'PRPF6']

In [102]:
distances = list()
instances = list(binarized_df.columns)

for n1 in instances:
    for n2 in instances:
        ds = skm.pairwise.distance.jaccard(binarized_df[n1], binarized_df[n2])
        distances.append((n1, n2, ds))

distances = pd.DataFrame(distances)
distances.head()

,0,1,2
0,PRODH,PRODH,0.000000
1,PRODH,PROK2,0.977528
2,PRODH,PROKR2,0.966102
3,PRODH,PROM1,0.986111
4,PRODH,PROP1,0.961039
